# Implementation of a 3D Unet Architecture


In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import os, time
from importlib import reload

import tensorflow as tf
import model, utilities

NameError: name 'Unet' is not defined

In [2]:
reload(utilities)
reload(model)

<module 'model' from 'c:\\Users\\Linus Meienberg\\Google Drive\\Janelia\\ImageSegmentation\\3D Unet\\model.py'>

In [3]:
# Fix for tensorflow-gpu issues that I found online... (don't ask me what it does)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
# Get permissible network input sized (cube lengths)
n_blocks = 2
valid_inputs = [n for n in range(512) if utilities.check_image_size(n, n_blocks=n_blocks)[0]]
print(valid_inputs)

[92, 100, 108, 116, 124, 132, 140, 148, 156, 164, 172, 180, 188, 196, 204, 212, 220, 228, 236, 244, 252, 260, 268, 276, 284, 292, 300, 308, 316, 324, 332, 340, 348, 356, 364, 372, 380, 388, 396, 404, 412, 420, 428, 436, 444, 452, 460, 468, 476, 484, 492, 500, 508]


In [5]:
unet = model.Unet()

In [6]:
unet(tf.ones((1,132,132,132,3)))

TypeError: call() takes 2 positional arguments but 3 were given

In [1]:
unet.summary()

NameError: name 'unet' is not defined